# Learning Manim myself by animating the Pythagoras Theorem

#### Install the Manim library

In [1]:
!pip install manim

In [4]:
from manim import *
#print(dir())

<code>dir()</code>: This built-in function returns a list of names in the current local scope or, if called with an object as an argument, it returns the names of the attributes and methods of that object.

#### Creating my First Scene
In Manim, animations are created by defining scenes within a class. Each scene is essentially a series of animations you want to play. Here’s how to create a simple scene with a circle.

#### Pythagoras Theorem Proof

In [5]:
%%manim -ql -p PythagoreanTheorem 
import numpy as np
from manim import *

class PythagoreanTheorem(Scene):
    def construct(self):
         # Create the triangle with vertices at scaled coordinates
        scale_factor = 0.5  # Adjust this value to change the size of the triangle
        A = np.array([0, 0, 0]) * scale_factor  # Vertex A (0, 0)
        B = np.array([0, 4, 0]) * scale_factor  # Vertex B (0, 4)
        C = np.array([3, 0, 0]) * scale_factor  # Vertex C (3, 0)

        triangle = Polygon(A, B, C, fill_color=BLUE, fill_opacity=0.5)
        triangle.shift(LEFT * 1.5, DOWN * 1)

        # Calculate the side lengths for the squares
        side_AB_length = np.linalg.norm(B - A)
        side_BC_length = np.linalg.norm(C - B)
        side_CA_length = np.linalg.norm(A - C)

        # Create squares aligned to each triangle side
        square_AB = Square(side_length=side_AB_length, fill_color=YELLOW, fill_opacity=0.5)
        square_AB.next_to(triangle, LEFT, buff=0)
        square_AB.rotate(Line(A, B).get_angle())

        square_BC = Square(side_length=side_BC_length, fill_color=GREEN, fill_opacity=0.5)
        square_BC.next_to(triangle, RIGHT, buff=0)
        square_BC.rotate(Line(B, C).get_angle())
        square_BC.shift(UP * 1.2, LEFT * 1.1, DOWN * 0.43, RIGHT * 0.123)

        square_CA = Square(side_length=side_CA_length, fill_color=RED, fill_opacity=0.5)
        square_CA.next_to(triangle, DOWN, buff=0)
        square_CA.rotate(Line(C, A).get_angle())

        # Create text labels using MathTex for LaTeX rendering
        label_a2 = MathTex("a^2").move_to(square1.get_center())
        label_b2 = MathTex("b^2").move_to(square2.get_center())
        label_c2 = MathTex("c^2").move_to(square3.get_center())
        theorem_text = Text("Pythagoras Theorem").to_edge(UP)
        equation_text = MathTex("a^2 + b^2 = c^2").next_to(square1, DOWN)

        self.play(Write(theorem_text))

        # Play the creation animations
        self.play(Create(triangle))
        self.play(Create(square1))
        self.play(Create(square2))
        self.play(Create(square3))

        # Play the text animations
        self.play(Write(label_a2), Write(label_b2), Write(label_c2))
        self.play(Write(equation_text))

        self.wait(2)


Manim Community v0.18.1

[10/27/24 16:53:23] INFO     Writing a^2 to media\Tex\4cd880482a26144d.tex                  ]8;id=9726;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=939016;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\tex_file_writing.py#109\109]8;;\

[10/27/24 16:53:25] INFO     Writing b^2 to media\Tex\252a52111768029a.tex                  ]8;id=863100;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=23127;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\tex_file_writing.py#109\109]8;;\

[10/27/24 16:53:26] INFO     Writing c^2 to media\Tex\e2b185d5b7067717.tex                  ]8;id=528079;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=337160;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\tex_file_writing.py#109\109]8;;\

[10/27/24 16:53:30] INFO     Writing a^2 + b^2 = c^2 to media\Tex\e4be163a00cf424f.tex      ]8;id=465978;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=793577;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\tex_file_writing.py#109\109]8;;\

...

                    INFO     Animation 7 : Partial movie file written in                   ]8;id=71775;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=878402;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'C:\Users\anoop\OneDrive\Desktop\Simple                                               
                             Projects\manim_pythagoras_theorem\media\videos\manim_pythagor                         
                             as_theorem\480p15\partial_movie_files\PythagoreanTheorem\6246                         
                             42324_1141282389_1977546982.mp4'                                                      

                    INFO     Combining to Movie file.                                      ]8;id=737153;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=332176;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py#617\617]8;;\

[10/27/24 16:53:38] INFO                                                                   ]8;id=332258;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=716969;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene_file_writer.py#737\737]8;;\
                             File ready at 'C:\Users\anoop\OneDrive\Desktop\Simple                                 
                             Projects\manim_pythagoras_theorem\media\videos\manim_pythagor                         
                             as_theorem\480p15\PythagoreanTheorem.mp4'                                             
                                                                                                                   

                    INFO     Rendered PythagoreanTheorem                                               ]8;id=163752;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=80766;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\scene\scene.py#247\247]8;;\
                             Played 8 animations                                                                   

                    INFO     Previewed File at: 'C:\Users\anoop\OneDrive\Desktop\Simple             ]8;id=36266;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\file_ops.py\file_ops.py]8;;\:]8;id=293306;file://C:\Users\anoop\AppData\Local\Programs\Python\Python312\Lib\site-packages\manim\utils\file_ops.py#231\231]8;;\
                             Projects\manim_pythagoras_theorem\media\videos\manim_pythagoras_theore                
                             m\480p15\PythagoreanTheorem.mp4'                                                      

The first line `%%manim -ql -p PythagoreanTheorem` is a magic command used in Jupyter notebooks to run a Manim scene directly from a notebook cell. Here’s a breakdown of what each part means:

### Breakdown of the Command:
- **`%%manim`**: This is a cell magic command specific to Jupyter notebooks that allows you to execute Manim code within the cell. When you run this cell, it tells Jupyter to process the contents using the Manim library.

- **`-ql`**: This stands for "quality low". It sets the rendering quality to low for quicker rendering times during development. In Manim, you can specify different quality settings:
  - **`-ql`**: Low quality 
  - **`-qm`**: Medium quality 
  - **`-qh`**: High quality 

- **`-p`**: This flag tells Manim to automatically play the rendered video once it's created, opening the video file in your default media player.

- **`PythagoreanTheorem`**: This is the name of the scene (class) to render.